In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("data.csv")

In [3]:
data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
0,63,1,1,145,233,1,2,150,0,2.3,3,0.0,6.0,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3.0,3.0,2
2,67,1,4,120,229,0,2,129,1,2.6,2,2.0,7.0,1
3,37,1,3,130,250,0,0,187,0,3.5,3,0.0,3.0,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0.0,3.0,0


In [4]:
data.describe()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
count,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,299.000000,301.000000,303.000000
mean,54.438944,0.679868,3.158416,131.689769,246.693069,0.148515,0.990099,149.607261,0.326733,1.039604,1.600660,0.672241,4.734219,0.937294
std,9.038662,0.467299,0.960126,17.599748,51.776918,0.356198,0.994971,22.875003,0.469794,1.161075,0.616226,0.937438,1.939706,1.228536
min,29.000000,0.000000,1.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,1.000000,0.000000,3.000000,0.000000
25%,48.000000,0.000000,3.000000,120.000000,211.000000,0.000000,0.000000,133.500000,0.000000,0.000000,1.000000,0.000000,3.000000,0.000000
50%,56.000000,1.000000,3.000000,130.000000,241.000000,0.000000,1.000000,153.000000,0.000000,0.800000,2.000000,0.000000,3.000000,0.000000
75%,61.000000,1.000000,4.000000,140.000000,275.000000,0.000000,2.000000,166.000000,1.000000,1.600000,2.000000,1.000000,7.000000,2.000000
max,77.000000,1.000000,4.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,3.000000,3.000000,7.000000,4.000000


In [5]:
data["num"] = data["num"].apply(lambda x: 1 if x > 0 else 0) 

In [6]:
data.isnull().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          4
thal        2
num         0
dtype: int64

## Looking For Corrs

In [7]:
corr_mat = data.corr()
corr_mat["num"].sort_values(ascending = False)

num         1.000000
thal        0.525689
ca          0.460442
exang       0.431894
oldpeak     0.424510
cp          0.414446
slope       0.339213
sex         0.276816
age         0.223120
restecg     0.169202
trestbps    0.150825
chol        0.085164
fbs         0.025264
thalach    -0.417167
Name: num, dtype: float64

## Dropping unnecessary cols

In [8]:
data.drop(["chol", "fbs"], inplace = True, axis = 1)

## Train Test Splitting

In [9]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(data, random_state = 42, test_size = 0.2)

## Pipeline

In [10]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

data = train_set.drop("num", axis = 1)
data_lbl = train_set["num"].copy()

pipe = Pipeline([
    ('imputer', SimpleImputer(strategy = "most_frequent")),
    ('scaler', StandardScaler())
])

In [11]:
pipe.fit(data)
scaled_data = pipe.transform(data)

In [12]:
scaled_data.shape

(242, 11)

## Model Selection

In [13]:
# from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

# model = KNeighborsClassifier(n_neighbors = 5)
model = RandomForestClassifier(n_estimators = 50, max_depth = 4)

model.fit(scaled_data, data_lbl)

RandomForestClassifier(max_depth=4, n_estimators=50)

In [14]:
from sklearn.model_selection import cross_val_score
import numpy as np

scores = cross_val_score(model, scaled_data, data_lbl, cv = 10, scoring = "accuracy")

In [15]:
with open("models.txt", "a") as f:
    f.write(f"\n{model}\n\t{scores.mean()}\n")

## Model Tuning

In [16]:
from sklearn.model_selection import GridSearchCV

params = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [4, 8, None]
}

grid = GridSearchCV(model, params, cv = 10)
grid.fit(scaled_data, data_lbl)

GridSearchCV(cv=10,
             estimator=RandomForestClassifier(max_depth=4, n_estimators=50),
             param_grid={'max_depth': [4, 8, None],
                         'n_estimators': [50, 100, 150, 200]})

In [17]:
grid.best_params_

{'max_depth': 8, 'n_estimators': 100}

In [18]:
with open("models.txt", "a") as f:
    f.write(f"\n\t{grid.best_score_}\n\t{grid.best_params_}\n")

## Final Model Testing

In [19]:
some_data = data.iloc[:5]
some_lbl = data_lbl.iloc[:5]

x_prep = pipe.transform(some_data)

In [20]:
model.predict(x_prep)

array([0, 0, 0, 0, 0])

In [21]:
some_lbl

132    0
202    0
196    0
75     0
176    0
Name: num, dtype: int64

## Testing on Test Data

In [22]:
x_test = test_set.drop("num", axis = 1)
y_test = test_set["num"].copy()

In [23]:
x_prep = pipe.transform(x_test)

In [24]:
model.predict(x_prep)

array([0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0,
       1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0])

In [25]:
y_test

179    0
228    1
111    1
246    1
60     1
      ..
249    0
104    1
300    1
193    1
184    1
Name: num, Length: 61, dtype: int64

In [26]:
cross_val_score(model, x_prep, y_test, cv = 10, scoring = "accuracy").mean()

np.float64(0.819047619047619)

## Saving Model

In [27]:
from joblib import dump

In [28]:
dump(model, 'Heart_Disease_Predictor.joblib')

['Heart_Disease_Predictor.joblib']